In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructField,StringType,IntegerType,DoubleType,StructType,TimestampType)
from pyspark.sql.functions import *
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('SparkByCredit').getOrCreate()

In [3]:
df = pd.read_json("cdw_sapp_credit.json",lines=True)

In [4]:
df.head()

,TRANSACTION_ID,DAY,MONTH,YEAR,CREDIT_CARD_NO,CUST_SSN,BRANCH_CODE,TRANSACTION_TYPE,TRANSACTION_VALUE
0,1,14,2,2018,4210653349028689,123459988,114,Education,78.90
1,2,20,3,2018,4210653349028689,123459988,35,Entertainment,14.24
2,3,8,7,2018,4210653349028689,123459988,160,Grocery,56.70
3,4,19,4,2018,4210653349028689,123459988,114,Entertainment,59.73
4,5,10,10,2018,4210653349028689,123459988,93,Gas,3.59


In [16]:
df.dtypes

TRANSACTION_ID         int64
DAY                    int64
MONTH                  int64
YEAR                   int64
CREDIT_CARD_NO         int64
CUST_SSN               int64
BRANCH_CODE            int64
TRANSACTION_TYPE      object
TRANSACTION_VALUE    float64
dtype: object

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46694 entries, 0 to 46693
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   TRANSACTION_ID     46694 non-null  int64  
 1   DAY                46694 non-null  int64  
 2   MONTH              46694 non-null  int64  
 3   YEAR               46694 non-null  int64  
 4   CREDIT_CARD_NO     46694 non-null  int64  
 5   CUST_SSN           46694 non-null  int64  
 6   BRANCH_CODE        46694 non-null  int64  
 7   TRANSACTION_TYPE   46694 non-null  object 
 8   TRANSACTION_VALUE  46694 non-null  float64
dtypes: float64(1), int64(7), object(1)
memory usage: 3.2+ MB


In [ ]:

df = df.rename(columns={"CREDIT_CARD_NO":"CUST_CC_NO"})
df["TRANSACTION_TYPE"] = df["TRANSACTION_TYPE"].astype(str)
df["TRANSACTION_VALUE"] = df["TRANSACTION_VALUE"].astype(float)
df["CUST_CC_NO"] = df["CUST_CC_NO"].astype(str)
df['date'] = pd.to_datetime(df[['YEAR','MONTH','DAY']], format='%d-%m-%Y')
df['TIMEID'] = df['date'].dt.strftime('%Y%m%d')
df = df.drop(['DAY',"MONTH","YEAR"], axis=1)
df = df.drop("date",axis=1)
df




In [12]:
df_top10 = df.head(10)
df_top10

,TRANSACTION_ID,CUST_CC_NO,CUST_SSN,BRANCH_CODE,TRANSACTION_TYPE,TRANSACTION_VALUE,TIMEID
0,1,4210653349028689,123459988,114,Education,78.90,20180214
1,2,4210653349028689,123459988,35,Entertainment,14.24,20180320
2,3,4210653349028689,123459988,160,Grocery,56.70,20180708
3,4,4210653349028689,123459988,114,Entertainment,59.73,20180419
4,5,4210653349028689,123459988,93,Gas,3.59,20181010
5,6,4210653349028689,123459988,164,Education,6.89,20180528
6,7,4210653349028689,123459988,119,Entertainment,43.39,20180519
7,8,4210653349028689,123459988,23,Gas,95.39,20180808
8,9,4210653349028689,123459988,166,Entertainment,93.26,20180318
9,10,4210653349028689,123459988,83,Bills,100.38,20180903


In [13]:
spark_credit_df = spark.createDataFrame(df)

C:\Spark\python\pyspark\sql\pandas\conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [14]:
spark_credit_df.show()

+--------------+----------------+---------+-----------+----------------+-----------------+--------+
|TRANSACTION_ID|      CUST_CC_NO| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|  TIMEID|
+--------------+----------------+---------+-----------+----------------+-----------------+--------+
|             1|4210653349028689|123459988|        114|       Education|             78.9|20180214|
|             2|4210653349028689|123459988|         35|   Entertainment|            14.24|20180320|
|             3|4210653349028689|123459988|        160|         Grocery|             56.7|20180708|
|             4|4210653349028689|123459988|        114|   Entertainment|            59.73|20180419|
|             5|4210653349028689|123459988|         93|             Gas|             3.59|20181010|
|             6|4210653349028689|123459988|        164|       Education|             6.89|20180528|
|             7|4210653349028689|123459988|        119|   Entertainment|            43.39|20180519|


In [8]:
spark_credit_df.write.format("jdbc").mode("append")\
        .option("url","jdbc:mysql://localhost:3306/creditcard_capstone")\
        .option("dbtable","creditcard_capstone.CDW_SAPP_CREDIT_CARD")\
        .option("user","root")\
        .option("password","password")\
        .save()